In [3]:
from datamodel import (
    Listing,
    Observation,
    Order,
    OrderDepth,
    ProsperityEncoder,
    UserId,
    Symbol,
    Trade,
    TradingState,
)
from typing import List
import copy
import numpy as np
import math
from itertools import permutations
import Round_3

###############
import sys
import os

sys.path.append(os.path.abspath("../"))

import webbrowser
from datetime import datetime
from pathlib import Path
from typing import Any, Optional
from importlib import reload
from collections import defaultdict
from functools import partial, reduce
from http.server import HTTPServer, SimpleHTTPRequestHandler

from prosperity3bt.data import has_day_data, read_day_data
from prosperity3bt.file_reader import (
    FileReader,
    FileSystemReader,
    PackageResourcesReader,
)
from prosperity3bt.models import BacktestResult
from prosperity3bt.runner import run_backtest
################


import json
from typing import Any

In [4]:
class tradable_product:
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"
    SQUID_INK = "SQUID_INK"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    PICNIC_BASKET1 = "PICNIC_BASKET1"
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    DJEMBES = "DJEMBES"
    VOLCANIC_ROCK = "VOLCANIC_ROCK"
    VOLCANIC_ROCK_VOUCHER_9500 = "VOLCANIC_ROCK_VOUCHER_9500"
    VOLCANIC_ROCK_VOUCHER_9750 = "VOLCANIC_ROCK_VOUCHER_9750"
    VOLCANIC_ROCK_VOUCHER_10000 = "VOLCANIC_ROCK_VOUCHER_10000"
    VOLCANIC_ROCK_VOUCHER_10250 = "VOLCANIC_ROCK_VOUCHER_10250"
    VOLCANIC_ROCK_VOUCHER_10500 = "VOLCANIC_ROCK_VOUCHER_10500"


class position_limit:
    KELP = 50
    RAINFOREST_RESIN = 50
    SQUID_INK = 50
    CROISSANTS = 250
    JAMS = 350
    PICNIC_BASKET1 = 60
    PICNIC_BASKET2 = 100
    DJEMBES = 60
    VOLCANIC_ROCK = 400
    VOLCANIC_ROCK_VOUCHER_9500 = 200
    VOLCANIC_ROCK_VOUCHER_9750 = 200
    VOLCANIC_ROCK_VOUCHER_10000 = 200
    VOLCANIC_ROCK_VOUCHER_10250 = 200
    VOLCANIC_ROCK_VOUCHER_10500 = 200

In [5]:
def parse_data(data_root: Optional[Path]) -> FileReader:
    if data_root is not None:
        return FileSystemReader(data_root)
    else:
        return PackageResourcesReader()


def parse_days(file_reader: FileReader, days: list[str]) -> list[tuple[int, int]]:
    parsed_days = []

    for arg in days:
        if "-" in arg:
            round_num, day_num = map(int, arg.split("-", 1))

            if not has_day_data(file_reader, round_num, day_num):
                print(f"Warning: no data found for round {round_num} day {day_num}")
                continue

            parsed_days.append((round_num, day_num))
        else:
            round_num = int(arg)

            parsed_days_in_round = []
            for day_num in range(-5, 6):
                if has_day_data(file_reader, round_num, day_num):
                    parsed_days_in_round.append((round_num, day_num))

            if len(parsed_days_in_round) == 0:
                print(f"Warning: no data found for round {round_num}")
                continue

            parsed_days.extend(parsed_days_in_round)

    if len(parsed_days) == 0:
        print("Error: did not find data for any requested round/day")
        sys.exit(1)

    return parsed_days


def parse_out(out: Optional[Path], no_out: bool) -> Optional[Path]:
    if out is not None:
        return out

    if no_out:
        return None

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return Path.cwd() / "backtests" / f"{timestamp}.log"


def print_day_summary(result: BacktestResult) -> None:
    last_timestamp = result.activity_logs[-1].timestamp

    product_lines = []
    total_profit = 0

    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        product = row.columns[2]
        profit = row.columns[-1]

        product_lines.append(f"{product}: {profit:,.0f}")
        total_profit += profit

    print(*reversed(product_lines), sep="\n")
    print(f"Total profit: {total_profit:,.0f}")


def merge_results(
    a: BacktestResult,
    b: BacktestResult,
    merge_profit_loss: bool,
    merge_timestamps: bool,
) -> BacktestResult:
    sandbox_logs = a.sandbox_logs[:]
    activity_logs = a.activity_logs[:]
    trades = a.trades[:]

    if merge_timestamps:
        a_last_timestamp = a.activity_logs[-1].timestamp
        timestamp_offset = a_last_timestamp + 100
    else:
        timestamp_offset = 0

    sandbox_logs.extend([row.with_offset(timestamp_offset) for row in b.sandbox_logs])
    trades.extend([row.with_offset(timestamp_offset) for row in b.trades])

    if merge_profit_loss:
        profit_loss_offsets = defaultdict(float)
        for row in reversed(a.activity_logs):
            if row.timestamp != a_last_timestamp:
                break

            profit_loss_offsets[row.columns[2]] = row.columns[-1]

        activity_logs.extend(
            [
                row.with_offset(timestamp_offset, profit_loss_offsets[row.columns[2]])
                for row in b.activity_logs
            ]
        )
    else:
        activity_logs.extend(
            [row.with_offset(timestamp_offset, 0) for row in b.activity_logs]
        )

    return BacktestResult(a.round_num, a.day_num, sandbox_logs, activity_logs, trades)


def write_output(output_file: Path, merged_results: BacktestResult) -> None:
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with output_file.open("w+", encoding="utf-8") as file:
        file.write("Sandbox logs:\n")
        for row in merged_results.sandbox_logs:
            file.write(str(row))

        file.write("\n\n\nActivities log:\n")
        file.write(
            "day;timestamp;product;bid_price_1;bid_volume_1;bid_price_2;bid_volume_2;bid_price_3;bid_volume_3;ask_price_1;ask_volume_1;ask_price_2;ask_volume_2;ask_price_3;ask_volume_3;mid_price;profit_and_loss\n"
        )
        file.write("\n".join(map(str, merged_results.activity_logs)))

        file.write("\n\n\n\n\nTrade History:\n")
        file.write("[\n")
        file.write(",\n".join(map(str, merged_results.trades)))
        file.write("]")


def print_overall_summary(results: list[BacktestResult]) -> None:
    print("Profit summary:")

    total_profit = 0
    for result in results:
        last_timestamp = result.activity_logs[-1].timestamp

        profit = 0
        for row in reversed(result.activity_logs):
            if row.timestamp != last_timestamp:
                break

            profit += row.columns[-1]

        print(f"Round {result.round_num} day {result.day_num}: {profit:,.0f}")
        total_profit += profit

    print(f"Total profit: {total_profit:,.0f}")

    return total_profit


def format_path(path: Path) -> str:
    cwd = Path.cwd()
    if path.is_relative_to(cwd):
        return str(path.relative_to(cwd))
    else:
        return str(path)


class HTTPRequestHandler(SimpleHTTPRequestHandler):
    def do_GET(self):
        self.server.shutdown_flag = True
        return super().do_GET()

    def end_headers(self) -> None:
        self.send_header("Access-Control-Allow-Origin", "*")
        return super().end_headers()

    def log_message(self, format: str, *args: Any) -> None:
        return


class CustomHTTPServer(HTTPServer):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.shutdown_flag = False


def open_visualizer(output_file: Path) -> None:
    http_handler = partial(HTTPRequestHandler, directory=str(output_file.parent))
    http_server = CustomHTTPServer(("localhost", 0), http_handler)

    webbrowser.open(
        f"https://jmerle.github.io/imc-prosperity-3-visualizer/?open=http://localhost:{http_server.server_port}/{output_file.name}"
    )

    while not http_server.shutdown_flag:
        http_server.handle_request()

In [6]:
all_data = []
file_reader = parse_data(
    Path(
        "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/prosperity3bt/resources"
    )
)
days = parse_days(file_reader, [str(day) for day in range(2, 3)])
for round_num, day_num in days:
    all_data.append(read_day_data(file_reader, round_num, day_num, no_names=True))

In [7]:
def main(all_data, params: dict, output_file=None) -> None:
    results = []
    file_reader = parse_data(
        Path(
            "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/prosperity3bt/resources"
        )
    )
    days = parse_days(file_reader, [str(day) for day in range(3, 4)])
    for i in range(len(days)):
        round_num, day_num = days[i]
        reload(Round_3)
        data = all_data[i]
        result = run_backtest(
            trader=Round_3.Trader(params),
            data=data,
            file_reader=file_reader,
            round_num=round_num,
            day_num=day_num,
            print_output=False,
            trade_matching_mode="all",
            no_names=True,  # args.no_names,
            show_progress_bar=False,
        )

        # print_day_summary(result)
        if len(days) > 1:
            print()

        results.append(result)

    if output_file is not None:
        merged_results = reduce(
            lambda a, b: merge_results(
                a, b, merge_profit_loss=True, merge_timestamps=True
            ),
            results,
        )
        write_output(output_file, merged_results)
        print(f"\nSuccessfully saved backtest results to {format_path(output_file)}")
        open_visualizer(output_file)

    final_profit = print_overall_summary(results)
    return final_profit

In [51]:
params = {
    tradable_product.RAINFOREST_RESIN: {
        "ask_slip": 1,
        "bid_slip": -1,
        "decay": 0.1,
        "threshold": 0.0,
    },
    tradable_product.KELP: {
        "alpha": np.array([[-6.20579905e-04], [3.58373569e-05]]),
        "beta": np.array([[1.0, -0.91325515]]),
        "gamma": np.array([[0.00538188, -0.34236735], [0.00595464, -0.48509457]]),
        "decay": 0.6,
        "threshold": 0.8,
    },
    tradable_product.SQUID_INK: {
        "z_threshold": 1.7,
    },
    tradable_product.PICNIC_BASKET1: {
        tradable_product.CROISSANTS: 6,
        tradable_product.JAMS: 3,
        tradable_product.DJEMBES: 1,
        "alpha": np.array([[-0.00077897], [0.0]]),
        "beta": np.array([[1.0, -0.07278235]]),
        "gamma": np.array([[-0.13665529, 0.00551131], [0.46266129, -0.0553495]]),
        "z_threshold": 2.2,
        "z_threshold2": 2.0,
    },
    tradable_product.PICNIC_BASKET2: {
        tradable_product.CROISSANTS: 4,
        tradable_product.JAMS: 2,
        tradable_product.DJEMBES: 0,
        "alpha": np.array([[-0.00018354], [0.00021965]]),
        "beta": np.array([[1.0, -1.944405]]),
        "gamma": np.array([[-0.03897751, -0.00675144], [-0.0137392, -0.01803694]]),
        "z_threshold": 1.4,
        "z_threshold2": 2,
    },
    tradable_product.CROISSANTS: {
        "alpha": np.array([[-0.00077897], [0.0]]),
        "beta": np.array([[1.0, -0.07278235]]),
        "gamma": np.array([[-0.13665529, 0.00551131], [0.46266129, -0.0553495]]),
        "z_threshold": 3.0,
    },
    tradable_product.JAMS: {
        "alpha": np.array([[0.0], [0.0]]),
        "beta": np.array([[1.0, -0.11138969]]),
        "gamma": np.array([[-0.05566674, 0.00370577], [0.19408842, -0.04752362]]),
        "z_threshold": 5.6,
    },
    tradable_product.DJEMBES: {
        "alpha": np.array([[0.0], [0.00141277]]),
        "beta": np.array([[1.0, -0.22807497]]),
        "gamma": np.array([[0.01875615, 0.00280019], [0.05341494, -0.04265734]]),
        "z_threshold": 3.6,
    },
    tradable_product.VOLCANIC_ROCK: {"pos": 0, "to_hedge": 0},
    tradable_product.VOLCANIC_ROCK_VOUCHER_9500: {
        "strike": 9500,
        "dte": 8e6,
        "z_threshold": 1.059128940308789,
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_9750: {
        "strike": 9750,
        "dte": 8e6,
        "z_threshold": 0.8600954832723592,
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_10000: {
        "strike": 10_000,
        "dte": 8e6,
        "z_threshold": 1.2651093313307173,
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_10250: {
        "strike": 10_250,
        "dte": 8e6,
        "z_threshold": 1.3923953674290663,
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_10500: {
        "strike": 10_500,
        "dte": 8e6,
        "z_threshold": 1.6777865342072642,
    },
}


result = main(
    all_data=all_data,
    params=params,
    output_file=Path(
        "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/Round 3/test.log"
    ),
)





Successfully saved backtest results to test.log
Profit summary:
Round 3 day 0: 107,508
Round 3 day 1: 236,906
Round 3 day 2: 115,716
Total profit: 460,129


In [ ]:
# param_range = np.arange(-1, 1, 0.25)
# all_combi = list(permutations(param_range, 2))

best = 0
perm = {"PNL": [], "i": [], "j": []}

print(f"Permuations Count: {len(list(permutations(np.arange(1, 3, 0.2), 2)))}")

for i in np.arange(2, 4, 0.2):
    # for j in np.arange(1.6, 2.4, 0.2):
    j = 3
    window = 0.3
    threshold = 0
    params = {
        tradable_product.RAINFOREST_RESIN: {
            "ask_slip": 1,
            "bid_slip": -1,
            "decay": 0.1,
            "threshold": 0.0,
        },
        tradable_product.KELP: {
            "alpha": np.array([[-6.20579905e-04], [3.58373569e-05]]),
            "beta": np.array([[1.0, -0.91325515]]),
            "gamma": np.array([[0.00538188, -0.34236735], [0.00595464, -0.48509457]]),
            "decay": 0.6,
            "threshold": 0.8,
        },
        tradable_product.SQUID_INK: {
            "z_threshold": 1.7,
        },
        tradable_product.PICNIC_BASKET1: {
            tradable_product.CROISSANTS: 6,
            tradable_product.JAMS: 3,
            tradable_product.DJEMBES: 1,
            "alpha": np.array([[-0.00077897], [0.0]]),
            "beta": np.array([[1.0, -0.07278235]]),
            "gamma": np.array([[-0.13665529, 0.00551131], [0.46266129, -0.0553495]]),
            "z_threshold": 2.2,
            "z_threshold2": 2.0,
        },
        tradable_product.PICNIC_BASKET2: {
            tradable_product.CROISSANTS: 4,
            tradable_product.JAMS: 2,
            tradable_product.DJEMBES: 0,
            "alpha": np.array([[-0.00018354], [0.00021965]]),
            "beta": np.array([[1.0, -1.944405]]),
            "gamma": np.array([[-0.03897751, -0.00675144], [-0.0137392, -0.01803694]]),
            "z_threshold": 1.4,
            "z_threshold2": 2,
        },
        tradable_product.CROISSANTS: {
            "alpha": np.array([[-0.00077897], [0.0]]),
            "beta": np.array([[1.0, -0.07278235]]),
            "gamma": np.array([[-0.13665529, 0.00551131], [0.46266129, -0.0553495]]),
            "z_threshold": i,
        },
        tradable_product.JAMS: {
            "alpha": np.array([[0.0], [0.0]]),
            "beta": np.array([[1.0, -0.11138969]]),
            "gamma": np.array([[-0.05566674, 0.00370577], [0.19408842, -0.04752362]]),
            "z_threshold": i,
        },
        tradable_product.DJEMBES: {
            "alpha": np.array([[0.0], [0.00141277]]),
            "beta": np.array([[1.0, -0.22807497]]),
            "gamma": np.array([[0.01875615, 0.00280019], [0.05341494, -0.04265734]]),
            "z_threshold": i,
        },
        tradable_product.VOLCANIC_ROCK: {"pos": 0, "to_hedge": 0},
        tradable_product.VOLCANIC_ROCK_VOUCHER_9500: {
            "strike": 9500,
            "dte": 8e6,
            "z_threshold": i,
            "exit_threshold": i,
        },
        tradable_product.VOLCANIC_ROCK_VOUCHER_9750: {
            "strike": 9750,
            "dte": 8e6,
            "z_threshold": i,
            "exit_threshold": 0.574145,
        },
        tradable_product.VOLCANIC_ROCK_VOUCHER_10000: {
            "strike": 10_000,
            "dte": 8e6,
            "z_threshold": i,
            "exit_threshold": 0.646979,
        },
        tradable_product.VOLCANIC_ROCK_VOUCHER_10250: {
            "strike": 10_250,
            "dte": 8e6,
            "z_threshold": i,
            "exit_threshold": 0.692927,
        },
        tradable_product.VOLCANIC_ROCK_VOUCHER_10500: {
            "strike": 10_500,
            "dte": 8e6,
            "z_threshold": i,
            "exit_threshold": 0.973857,
        },
    }

    result = main(all_data=all_data, params=params)

    perm["PNL"].append(result)
    perm["i"].append(i)
    perm["j"].append(j)

    if result > best:
        best = result
        print(best, i, j)

print(best)


Permuations Count: 90



Profit summary:
Round 3 day 0: -10,879
Round 3 day 1: -11,563
Round 3 day 2: -11,744
Total profit: -34,186



Profit summary:
Round 3 day 0: -6,723
Round 3 day 1: -11,631
Round 3 day 2: -15,514
Total profit: -33,868



Profit summary:
Round 3 day 0: -4,550
Round 3 day 1: -8,752
Round 3 day 2: -12,404
Total profit: -25,706



Profit summary:
Round 3 day 0: 2,537
Round 3 day 1: -4,500
Round 3 day 2: -12,046
Total profit: -14,008



Profit summary:
Round 3 day 0: 1,319
Round 3 day 1: -2,986
Round 3 day 2: -8,346
Total profit: -10,014



Profit summary:
Round 3 day 0: 1,576
Round 3 day 1: -3,567
Round 3 day 2: -4,468
Total profit: -6,458



Profit summary:
Round 3 day 0: 1,344
Round 3 day 1: 1,715
Round 3 day 2: -2,008
Total profit: 1,051
1051.0 3.200000000000001 3



Profit summary:
Round 3 day 0: 2,426
Round 3 day 1: 1,834
Round 3 day 2: -3,150
Total profit: 1,110
1110.0 3.4000000000000012 3



Profit summary:
Round 3 day 0: 2,610
Round 3 day 1: 1,779
Round 3 day

In [38]:
import pandas as pd

pd.DataFrame.from_dict(perm).sort_values(by="PNL", ascending=False)

,PNL,i,j
2,64007.5,1.4,2.0
3,61277.0,1.4,2.2
6,57898.0,1.6,2.0
1,57369.5,1.4,1.8
19,53686.0,2.2,2.2
15,52732.0,2.0,2.2
5,51718.0,1.6,1.8
0,50774.0,1.4,1.6
17,50213.0,2.2,1.8
18,49752.0,2.2,2.0
